In [16]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import warnings
%matplotlib inline
warnings.filterwarnings(action='ignore')

def changes_table(trade):
        
    # Making whole total file
    total_path = 'data/total/'
    fl = 'data/total/total.csv'
    total_file = os.path.exists(fl)
    
    # If there is already 'total.csv', you need to remove it first. If you don't do this, the 'total.csv' file will be appended by new command.
    if total_file:
        os.remove(fl)
        
    file_list = os.listdir(total_path)
    all_files = [file for file  in file_list if file.endswith('.csv')]

    df = pd.DataFrame()

    for i in all_files:
        data = pd.read_csv(total_path + i, index_col=0)
        if len(data.index) == 1:
            pass
        else:
            df = df.append(data)
    df.reset_index(drop = True, inplace=True)
    df.to_csv(total_path + 'total.csv')
    
    # Remove Asean & EU-28 data
    no_asean = df['Reporter'] != 'ASEAN' # code = 975
    no_eu = df['Reporter'] != 'EU-28' # code = 97
    no_other_asia = df['Reporter'] != 'Other Asia, nes' # code = 490
    df = df[no_asean & no_eu & no_other_asia]
    
    if trade == 1:
        df_import = df[df['Trade Flow']=='Import']
        df_import = df_import.groupby('Year').sum('Netweight (kg)')
        df_import.reset_index(inplace=True)
        df_import_amount = df_import[['Year', 'Netweight (kg)']]
        amount_val = df_import_amount['Netweight (kg)'].values
        amount_changes = amount_val[1:]/amount_val[:-1]
        changes = np.round(amount_changes * 100 - 100)
        changes = [0.0] + list(changes)
        df_import_amount['Import Changes'] = changes
        return df_import_amount
    
    elif trade == 2:
        df_export = df[df['Trade Flow']=='Export']
        df_export = df_export.groupby('Year').sum('Netweight (kg)')
        df_export.reset_index(inplace=True)
        df_export_amount = df_export[['Year', 'Netweight (kg)']]
        amount_val = df_export_amount['Netweight (kg)'].values
        amount_changes = amount_val[1:]/amount_val[:-1]
        changes = np.round(amount_changes * 100 - 100)
        changes = [0.0] + list(changes)
        df_export_amount['Export Changes'] = changes
        return df_export_amount
    
    else:
        print('Select trade number: Import=1, Export=2')
        
    return


def rank_table(years, trade, top):
    
    # Making whole total file
    total_path = 'data/total/'
    fl = 'data/total/total.csv'
    total_file = os.path.exists(fl)
    
    # If there is already 'total.csv', you need to remove it first. If you don't do this, the 'total.csv' file will be appended by new command.
    if total_file:
        os.remove(fl)
        
    file_list = os.listdir(total_path)
    all_files = [file for file  in file_list if file.endswith('.csv')]

    df = pd.DataFrame()

    for i in all_files:
        data = pd.read_csv(total_path + i, index_col=0)
        if len(data.index) == 1:
            pass
        else:
            df = df.append(data)
            
    df.reset_index(drop = True, inplace=True)
    df.to_csv(total_path + 'total.csv')
    
    # Remove Asean & EU-28 data
    df = pd.read_csv(fl, index_col=0)
    no_asean = df['Reporter'] != 'ASEAN' # code = 975
    no_eu = df['Reporter'] != 'EU-28' # code = 97
    no_other_asia = df['Reporter'] != 'Other Asia, nes' # code = 490
    df = df[no_asean & no_eu & no_other_asia]
    df = df.astype({'Year': 'string'})
    
    if trade == 1:
        df_import_top = pd.DataFrame()
        df_import = df[df['Trade Flow']=='Import']

        df_import = df_import.groupby(['Year', 'Reporter']).sum('Netweight (kg)')
        df_import.reset_index(inplace=True)
        
        for i in years:
        
            df_import_year = df_import[df_import['Year']==i]
            
            df_import_year['Percentage'] = df_import_year['Netweight (kg)']/df_import_year['Netweight (kg)'].sum()
            df_import_year.sort_values(by='Netweight (kg)', ascending=False, inplace=True)
            df_import_year_top = df_import_year[0:top]
            df_import_year_top = df_import_year_top[['Reporter']]
            df_import_year_top.reset_index(inplace=True)
            
            df_import_year_top.drop('index', axis=1, inplace=True)
            df_import_year_top.rename(columns={'Reporter':i+'_Import'}, inplace=True)
            df_import_top = pd.concat([df_import_top, df_import_year_top], axis=1)
        
        return df_import_top

    elif trade == 2:
        df_export_top = pd.DataFrame()
        df_export = df[df['Trade Flow']=='Export']

        df_export = df_export.groupby(['Year', 'Reporter']).sum('Netweight (kg)')
        df_export.reset_index(inplace=True)

        for i in years:

            df_export_year = df_export[df_export['Year']==i]

            df_export_year['Percentage'] = df_export_year['Netweight (kg)']/df_export_year['Netweight (kg)'].sum()
            df_export_year.sort_values(by='Netweight (kg)', ascending=False, inplace=True)
            df_export_year_top = df_export_year[0:top]
            df_export_year_top = df_export_year_top[['Reporter']]
            df_export_year_top.reset_index(inplace=True)

            df_export_year_top.drop('index', axis=1, inplace=True)
            df_export_year_top.rename(columns={'Reporter':i+'_Export'}, inplace=True)
            df_export_top = pd.concat([df_export_top, df_export_year_top], axis=1)
            
        return df_export_top
    
    else:
        print('Select trade number: Import=1, Export=2')
        
    return